In [ ]:
%load_ext autoreload
%autoreload 2

from nanopyx.data.examples.download import ExampleDataManager
from nanopyx.core.transform.binning import rebin2d
from nanopyx.core.transform.image_magnify import fourier_zoom, catmull_rom_zoom
from scipy.ndimage import zoom as scipy_zoom
from skimage.transform import (
    rescale as ski_rescale, 
    resize as ski_resize, 
    downscale_local_mean as ski_downscale_local_mean
    )

downloader = ExampleDataManager()
z = downloader.get_zarr("SMLMS2013_HDTubulinAlexa647")
image = z['data'][0]
imageHalfSize = rebin2d(image, 2, mode="mean")

In [ ]:
%%prun -s cumulative -q -l 1000 -T prun_magnify
# Profile the run

imageFFTZoom = fourier_zoom(imageHalfSize, 2)
imageCRZoom = catmull_rom_zoom(imageHalfSize, 2)
imageScipyZoom = scipy_zoom(imageHalfSize, 2)
imageSkiZoom = ski_rescale(imageHalfSize, 2, anti_aliasing=False)

In [ ]:
# Show profiling
print(open('prun_magnify', 'r').read())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Create a figure with 3 subplots arranged in a row
fig, axs = plt.subplots(5, 2, figsize=(16, 2*16))
sns.heatmap(image, ax=axs[0,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(imageHalfSize, ax=axs[0,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[0,0].set_title("orginal")
axs[0,1].set_title("half-size")

sns.heatmap(imageFFTZoom, ax=axs[1,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(image-imageFFTZoom, ax=axs[1,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[1,0].set_title("fft-zoom")
axs[1,1].set_title("fft-zoom-error")

sns.heatmap(imageCRZoom, ax=axs[2,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(image-imageCRZoom, ax=axs[2,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[2,0].set_title("cr-zoom")
axs[2,1].set_title("cr-zoom-error")

sns.heatmap(imageScipyZoom, ax=axs[3,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(image-imageScipyZoom, ax=axs[3,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[3,0].set_title("scipy-zoom")
axs[3,1].set_title("scipy-zoom-error")

sns.heatmap(imageSkiZoom, ax=axs[4,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(image-imageSkiZoom, ax=axs[4,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[4,0].set_title("skimage-zoom")
axs[4,1].set_title("skimage-zoom-error")

# Show the figure
plt.show()
